In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler

In [ ]:
# Retrieve Training and Testing Data 
df_train = pd.read_csv('data/trainClean.csv')
df_test = pd.read_csv('data/testClean.csv')

In [ ]:
X_train = df_train.drop(columns='Transported')
y_train = df_train['Transported']

In [ ]:
lr = LogisticRegressionCV(n_jobs=-1)
lr.fit(X_train, y_train)

pred = lr.predict(df_test)
pred

In [ ]:
tfMap = { 0: False, 1: True }

kaggleCSV = pd.DataFrame(columns=['PassengerId', 'Transported'])
kaggleCSV['PassengerId'] = df_test['PassengerId']
kaggleCSV['Transported'] = [tfMap[i] for i in pred]

kaggleCSV.to_csv('results/example_results.csv', index=False)